In [ ]:
#creditset2.csv을 df로 읽으세요.
import pandas as pd
df = pd.read_csv('creditset2.csv')
df.head()

#default10yr에 대해 value_counts 해보세요
df.default10yr.value_counts()/2000

#default10yr에 따라 다른 변수 값 평균 확인하세요.
df.groupby("default10yr").mean()

In [ ]:
#data.csv pass.fail을 target, 분류모델링

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
df = pd.read_csv("data.csv")
#df = pd.get_dummies( df, columns=["rank"])
X = df.drop("Pass.Fail", axis=1)
y = df["Pass.Fail"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,random_state=0)
model = DecisionTreeClassifier(random_state=0)
model.fit( X_train, y_train )
y_pred = model.predict( X_test )   #model.predict_proba( X)
model.score(X_test,y_test)   #Accuracy, 정분류율
print(classification_report( y_test, y_pred  ))


In [ ]:
#diabetes.csv, outcome이 y변수,
from imblearn.over_sampling import SMOTE
df = pd.read_csv("diabetes.csv")
X = df.drop("Outcome", axis=1)
y = df["Outcome"]
#Oversampling 0.4로 지정해서 다시 모델링해보세요
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y,random_state=0)
sm = SMOTE(random_state=0, sampling_strategy=0.7, k_neighbors=5)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

model = DecisionTreeClassifier(random_state=0)
model.fit( X_resampled, y_resampled )
y_pred = model.predict( X_test )   #model.predict_proba( X)
model.score(X_test,y_test)   #Accuracy, 정분류율
print(classification_report( y_test, y_pred  ))

In [73]:
import pickle
with open( "model.pkl", "wb" ) as f:
      pickle.dump( model, f )

In [ ]:
with open( "model.pkl", "rb") as f:
  model2 = pickle.load( f )
model2.predict( X_test )

In [79]:
pd.DataFrame( model2.feature_importances_,
             index=X.columns).sort_values(0, ascending=False)

,0
Glucose,0.287535
BMI,0.221041
Age,0.129981
DiabetesPedigreeFunction,0.124923
BloodPressure,0.111677
SkinThickness,0.053242
Pregnancies,0.043591
Insulin,0.028011


In [ ]:
#mortDefault.zip 업로드-> unzip
#2007년도 자료를 읽으세요
#default: y
# creditScore, ccDebt,yearsEmploy, houseAge: X
#year: 삭제, id: 삭제
#default==0 ->default가 1인 행의 수만큼 sample
#default1과 sample 데이터프레임 합치셔서 분류모델링.
!unzip mortDefault.zip

In [ ]:
mort = pd.read_csv("mortDefault2007.csv")
mort.drop("year", axis=1, inplace=True)
mort1 = mort.query( "default==1")
mort0 = mort.query( "default==0")
mort0 = mort0.sample(2060)
mort2 = pd.concat( [mort1, mort0])

X = mort2.drop("default", axis=1)
y = mort2.default

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify = y,
                                                    random_state=42)
model = DecisionTreeClassifier(max_depth=7).fit( X_train, y_train) #pruning
pred = model.predict( X_test)
print(classification_report(y_test, pred))

In [100]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(estimator=DecisionTreeClassifier(),
                  param_grid = {"max_depth":[3,5,7]},
                  cv=5,
                  )
gs.fit( X_train, y_train)
gs.best_params_
gs.best_score_
model2 = gs.best_estimator_

In [96]:
#2008년도 자료를 읽으셔서-> 부도 행들만 선택하셔서 -> 모형예측, 결과를 확인
mort2008 = pd.read_csv("mortDefault2008.csv")
mort2008.drop("year", axis=1, inplace=True)
mort2008_2 = mort2008.query( "default==1")
pred = model.predict( mort2008_2.drop("default", axis=1))
print(classification_report(mort2008_2.default, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.79      0.88     14811

    accuracy                           0.79     14811
   macro avg       0.50      0.39      0.44     14811
weighted avg       1.00      0.79      0.88     14811



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [107]:
#pykrx 에서 특정 종목의 특정 기간의 자료를 다운로드
#sp500_tm.csv -> 분류 모델링, Direction: y변수,
#!pip install pykrx -qqq
#등락률 > 0 1, 아니면 0으로 변환 -> y
from pykrx import stock
df = stock.get_market_ohlcv_by_date(ticker="005930", fromdate="20230101", todate="20240630")
#종가(t-1), 거래량(t-1) : X
df['종가(t-1)'] = df['종가'].shift(1)
df['종가(t-2)'] = df['종가'].shift(2)
df['거래량(t-1)'] = df['거래량'].shift(1)
df = df.dropna()
df['y'] = (df['등락률']>0).astype(int)
df1 = df.iloc[:, 6:10]
#분류모델링 RandomForest, GridSearch
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X = df1.drop("y", axis=1)
y = df1.y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify = y, random_state=42)
gs =  GridSearchCV(estimator=RandomForestClassifier(),
                  param_grid = {"max_depth":[3,5,7], "n_estimators":[50,100]},
                  cv=5)
gs.fit( X_train, y_train)
gs.best_score_

np.float64(0.5601987142022209)

In [111]:
#sp500_tm.csv -> 분류 모델링, Direction: y변수
df = pd.read_csv("sp500_tm.csv")
df["avgPos_t-1"] = df.avgPos.shift(1)
df["avgNeg_t-1"] = df.avgNeg.shift(1)
df.dropna(inplace=True)

In [113]:
X_train, X_test, y_train, y_test = train_test_split(df.drop("Direction", axis=1),
                                                    df.Direction, test_size=0.1, random_state=42)
gs =  GridSearchCV(estimator=RandomForestClassifier(),
                  param_grid = {"max_depth":[5,7,9], "n_estimators":[100,200]},
                  cv=5)
gs.fit( X_train, y_train)
gs.best_score_

np.float64(0.5607843137254902)

In [118]:
#sms.csv -> type: y변수, 나머지: X변수
# RandomForest, GradientBoosting, DecisionTree
# Gridsearch X

df = pd.read_csv("sms.csv")
X = df.drop("type", axis=1)
y = df["type"]
df["type"].value_counts()/5574

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify = y,
                                                    random_state=42)
models = [DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier()]
for model in models:
  model.fit(X_train, y_train)
  pred = model.predict(X_test)
  print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96       966
           1       0.86      0.64      0.74       149

    accuracy                           0.94      1115
   macro avg       0.90      0.81      0.85      1115
weighted avg       0.94      0.94      0.93      1115

              precision    recall  f1-score   support

           0       0.95      0.98      0.97       966
           1       0.86      0.66      0.75       149

    accuracy                           0.94      1115
   macro avg       0.91      0.82      0.86      1115
weighted avg       0.94      0.94      0.94      1115

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       966
           1       0.94      0.45      0.61       149

    accuracy                           0.92      1115
   macro avg       0.93      0.72      0.78      1115
weighted avg       0.92      0.92      0.91      1115



In [123]:

# creditset2.csv를 이용해서 분류모형을 만들고 pkl 파일로 저장하세요

df = pd.read_csv("creditset2.csv")
X = df.drop("default10yr", axis=1)
y = df.default10yr

model = DecisionTreeClassifier().fit(X,y)

import pickle
with open( "model.pkl", "wb" ) as f:
      pickle.dump( model, f )

with open( "model.pkl", "rb" ) as f:
    loaded_model = pickle.load(f)


# 어제 fastapi노트에서 불러서 api로 예측값을 제시하도록 만들어보세요.
X.columns

Index(['income', 'age', 'loan'], dtype='object')

In [ ]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [134]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape
X_train_1 = X_train.reshape(  60000, 28*28)  #2d
X_test_1 = X_test.reshape(  10000, 28*28)  #2d

model2 = GradientBoostingClassifier()
model2.fit( X_train_1,y_train)#model2.fit( X_train_1, pd.get_dummies(y_train))

KeyboardInterrupt: 

In [ ]:
pred = model2.predict( X_test_1)
print( classification_report( y_test, pred))